In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import os
from sklearn.model_selection import train_test_split
# Any results you write to the current directory are saved as output.

In [2]:
# !pip install iterative-stratification

In [3]:
os.listdir(r'C:\Users\Kaggle\BengaliAI\inputs')

['bengaliai', 'bengaliai-cv19', 'bengaliai-cv19.zip', 'bengaliai.zip']

Add heng's code to our envionment and import the modules

In [4]:
hengs_path = r'C:\Users\Kaggle\BengaliAI\seResNext_gridmask'
sys.path.append(hengs_path)

In [5]:
from common  import *
from model   import *
from kaggle import *

matplotlib.get_backend :  module://ipykernel.pylab.backend_inline


Heng uses his own version of datasplit(which I have uploaded [here](https://www.kaggle.com/bibek777/hengdata)) in his codes. I tried using it but get memory error, maybe it's too large to load. So I have edited the dataloader in his code and use different split for train and valid dataset. The codes/ideas for dataloader is taken from this [kernel](https://www.kaggle.com/backaggle/catalyst-baseline). Also the dataset used in this kernel is taken from [here](https://www.kaggle.com/pestipeti/bengaliai), uploaded by Peter

In [6]:
train_df = pd.read_csv(r"C:\Users\Kaggle\BengaliAI\inputs\bengaliai-cv19/train.csv")
train_df['id'] = train_df['image_id'].apply(lambda x: int(x.split('_')[1]))

X, y = train_df[['id', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic']]\
.values[:,0], train_df.values[:,1:]

train_df['fold'] = np.nan

In [7]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
nfold = 7   #### change folds
seed=42
mskf = MultilabelStratifiedKFold(n_splits=nfold, random_state=seed,shuffle=True)
for i, (_, test_index) in enumerate(mskf.split(X, y)):
    train_df.iloc[test_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')

valid_df = train_df[train_df['fold']==4]   #### change folds
train_df = train_df[train_df['fold']!=4]   #### change folds

dropVar = ['id','fold']
train_df.drop(dropVar,1,inplace=True)
valid_df.drop(dropVar,1,inplace=True)

In [8]:
valid_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
9,Train_9,64,7,1,তেঁ
21,Train_21,60,4,0,ণ্টু
25,Train_25,61,1,0,ণ্ঠা
40,Train_40,32,2,0,ঙ্কি
41,Train_41,23,4,0,গু


In [9]:
data_root = r"C:\Users\Kaggle\BengaliAI\inputs\bengaliai\256_train\256"
TASK_NAME = [ 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic' ]
NUM_TASK = len(TASK_NAME)

In [10]:
import albumentations
from albumentations.core.transforms_interface import DualTransform
from albumentations.augmentations import functional as aa


class GridMask(DualTransform):

    def __init__(self, num_grid=3, fill_value=0, rotate=0, mode=0, always_apply=False, p=0.5):
        super(GridMask, self).__init__(always_apply, p)
        if isinstance(num_grid, int):
            num_grid = (num_grid, num_grid)
        if isinstance(rotate, int):
            rotate = (-rotate, rotate)
        self.num_grid = num_grid
        self.fill_value = fill_value
        self.rotate = rotate
        self.mode = mode
        self.masks = None
        self.rand_h_max = []
        self.rand_w_max = []

    def init_masks(self, height, width):
        if self.masks is None:
            self.masks = []
            n_masks = self.num_grid[1] - self.num_grid[0] + 1
            for n, n_g in enumerate(range(self.num_grid[0], self.num_grid[1] + 1, 1)):
                grid_h = height / n_g
                grid_w = width / n_g
                this_mask = np.ones((int((n_g + 1) * grid_h), int((n_g + 1) * grid_w))).astype(np.uint8)
                for i in range(n_g + 1):
                    for j in range(n_g + 1):
                        this_mask[
                             int(i * grid_h) : int(i * grid_h + grid_h / 2),
                             int(j * grid_w) : int(j * grid_w + grid_w / 2)
                        ] = self.fill_value
                        if self.mode == 2:
                            this_mask[
                                 int(i * grid_h + grid_h / 2) : int(i * grid_h + grid_h),
                                 int(j * grid_w + grid_w / 2) : int(j * grid_w + grid_w)
                            ] = self.fill_value
                
                if self.mode == 1:
                    this_mask = 1 - this_mask

                self.masks.append(this_mask)
                self.rand_h_max.append(grid_h)
                self.rand_w_max.append(grid_w)

    def apply(self, image, mask, rand_h, rand_w, angle, **params):
        h, w = image.shape[:2]
        mask = aa.rotate(mask, angle) if self.rotate[1] > 0 else mask
        mask = mask[:,:,np.newaxis] if image.ndim == 3 else mask
        image *= mask[rand_h:rand_h+h, rand_w:rand_w+w].astype(image.dtype)
        return image

#     def get_params_dependent_on_targets(self, params,img):
#         img = params['image']
    def get_params_dependent_on_targets(self,img):
        height, width = img.shape[:2]
        self.init_masks(height, width)

        mid = np.random.randint(len(self.masks))
        mask = self.masks[mid]
        rand_h = np.random.randint(self.rand_h_max[mid])
        rand_w = np.random.randint(self.rand_w_max[mid])
        angle = np.random.randint(self.rotate[0], self.rotate[1]) if self.rotate[1] > 0 else 0

        return {'mask': mask, 'rand_h': rand_h, 'rand_w': rand_w, 'angle': angle}

    @property
    def targets_as_params(self):
        return ['image']

    def get_transform_init_args_names(self):
        return ('num_grid', 'fill_value', 'rotate', 'mode')

In [11]:
class KaggleDataset(Dataset):

    def __init__(self, df, data_path, augment=None):
        self.image_ids = df['image_id'].values
        self.grapheme_roots = df['grapheme_root'].values
        self.vowel_diacritics = df['vowel_diacritic'].values
        self.consonant_diacritics = df['consonant_diacritic'].values

        self.data_path = data_path
        self.augment = augment

    def __str__(self):
        string  = ''
        string += '\tlen = %d\n'%len(self)
        string += '\n'
        return string


    def __len__(self):
        return len(self.image_ids)


    def __getitem__(self, index):
        # print(index)
        image_id = self.image_ids[index]
        grapheme_root = self.grapheme_roots[index]
        vowel_diacritic = self.vowel_diacritics[index]
        consonant_diacritic = self.consonant_diacritics[index]

        image_id = os.path.join(self.data_path, image_id + '.png')

        image = cv2.imread(image_id, 0)
        image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
        image = image.astype(np.float32)/255
        label = [grapheme_root, vowel_diacritic, consonant_diacritic]

        infor = Struct(
            index    = index,
            image_id = image_id,
        )

        if self.augment is None:
            return image, label, infor
        else:
            return self.augment(image, label, infor)

In [12]:
def null_collate(batch):
    batch_size = len(batch)

    input = []
    label = []
    infor = []
    for b in range(batch_size):
        input.append(batch[b][0])
        label.append(batch[b][1])
        infor.append(batch[b][-1])

    input = np.stack(input)
    #input = input[...,::-1].copy()
    input = input.transpose(0,3,1,2)

    label = np.stack(label)

    #----
    input = torch.from_numpy(input).float()
    truth = torch.from_numpy(label).long()
    truth0, truth1, truth2 = truth[:,0],truth[:,1],truth[:,2]
    truth = [truth0, truth1, truth2]
    return input, truth, infor


##############################################################

def tensor_to_image(tensor):
    image = tensor.data.cpu().numpy()
    image = image.transpose(0,2,3,1)
    #image = image[...,::-1]
    return image


##############################################################

def do_random_crop_rotate_rescale(
    image,
    mode={'rotate': 10,'scale': 0.1,'shift': 0.1}
):

    dangle = 0
    dscale_x, dscale_y = 0,0
    dshift_x, dshift_y = 0,0

    for k,v in mode.items():
        if   'rotate'== k:
            dangle = np.random.uniform(-v, v)
        elif 'scale' == k:
            dscale_x, dscale_y = np.random.uniform(-1, 1, 2)*v
        elif 'shift' == k:
            dshift_x, dshift_y = np.random.uniform(-1, 1, 2)*v
        else:
            raise NotImplementedError

    #----

    height, width = image.shape[:2]

    cos = np.cos(dangle/180*PI)
    sin = np.sin(dangle/180*PI)
    sx,sy = 1 + dscale_x, 1+ dscale_y #1,1 #
    tx,ty = dshift_x*width, dshift_y*height

    src = np.array([[-width/2,-height/2],[ width/2,-height/2],[ width/2, height/2],[-width/2, height/2]], np.float32)
    src = src*[sx,sy]
    x = (src*[cos,-sin]).sum(1)+width/2 +tx
    y = (src*[sin, cos]).sum(1)+height/2+ty
    src = np.column_stack([x,y])

    dst = np.array([[0,0],[width,0],[width,height],[0,height]])
    s = src.astype(np.float32)
    d = dst.astype(np.float32)
    transform = cv2.getPerspectiveTransform(s,d)
    image = cv2.warpPerspective( image, transform, (width, height), flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT, borderValue=(1,1,1))

    return image


def do_random_log_contast(image, gain=[0.70, 1.30] ):
    gain = np.random.uniform(gain[0],gain[1],1)
    inverse = np.random.choice(2,1)

    if inverse==0:
        image = gain*np.log(image+1)
    else:
        image = gain*(2**image-1)

    image = np.clip(image,0,1)
    return image


#https://github.com/albumentations-team/albumentations/blob/8b58a3dbd2f35558b3790a1dbff6b42b98e89ea5/albumentations/augmentations/transforms.py
def do_grid_distortion(image, distort=0.25, num_step = 10):

    # http://pythology.blogspot.sg/2014/03/interpolation-on-regular-distorted-grid.html
    distort_x = [1 + random.uniform(-distort,distort) for i in range(num_step + 1)]
    distort_y = [1 + random.uniform(-distort,distort) for i in range(num_step + 1)]

    #---
    height, width = image.shape[:2]
    xx = np.zeros(width, np.float32)
    step_x = width // num_step

    prev = 0
    for i, x in enumerate(range(0, width, step_x)):
        start = x
        end   = x + step_x
        if end > width:
            end = width
            cur = width
        else:
            cur = prev + step_x * distort_x[i]
        xx[start:end] = np.linspace(prev, cur, end - start)
        prev = cur


    yy = np.zeros(height, np.float32)
    step_y = height // num_step

    prev = 0
    for idx, y in enumerate(range(0, height, step_y)):
        start = y
        end = y + step_y
        if end > height:
            end = height
            cur = height
        else:
            cur = prev + step_y * distort_y[idx]

        yy[start:end] = np.linspace(prev, cur, end - start)
        prev = cur

    map_x, map_y = np.meshgrid(xx, yy)
    map_x = map_x.astype(np.float32)
    map_y = map_y.astype(np.float32)
    image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(1,1,1))

    return image



# ##---
# #https://github.com/chainer/chainercv/blob/master/chainercv/links/model/ssd/transforms.py
def do_random_contast(image, alpha=[0,1]):
    beta  = 0
    alpha = random.uniform(*alpha) + 1
    image = image.astype(np.float32) * alpha + beta
    image = np.clip(image,0,1)
    return image

def do_mixup(input, onehot,truth):
    batch_size = len(input)

    alpha = 0.4  #0.2,0.4
    gamma = np.random.beta(alpha, alpha)
    gamma = max(1-gamma,gamma)

    # #mixup https://github.com/moskomule/mixup.pytorch/blob/master/main.py
    perm = torch.randperm(batch_size).to(input.device)
    perm_input  = input[perm]
    perm_onehot = [t[perm] for t in onehot]
    mix_input  = gamma*input + (1-gamma)*perm_input
    mix_onehot = [gamma*t    + (1-gamma)*perm_t for t,perm_t in zip(truth,perm_onehot)]
    return mix_input, mix_onehot


In [13]:
def doGridmask(image,num_grid,rotate):
    Gridmask=GridMask(num_grid=num_grid, fill_value=0, rotate=rotate, mode=0, always_apply=False, p=1)
    out=Gridmask.get_params_dependent_on_targets(image)
    mask = out['mask']
    rand_h = out['rand_h']
    rand_w = out['rand_w']
    angle = out['angle']
    image=Gridmask.apply(image,mask,rand_h,rand_w,angle)
    return image

In [14]:
################################################################################################

def train_augment(image, label, infor):
    if np.random.rand()< 0.3:
        image = do_random_crop_rotate_rescale(image, mode={'rotate': 17.5,'scale': 0.25,'shift': 0.08})
    else:
        image = doGridmask(image,5,25)
    return image, label, infor

def valid_augment(image, label, infor):
    return image, label, infor

In [15]:
from albumentations.augmentations.transforms import *
# train_augment = albumentations.Compose([GridMask(num_grid=3, p=1),Rotate(5,p=0.1),OpticalDistortion(p=0.1)])

In [16]:
#------------------------------------
def do_valid(net, valid_loader, out_dir=None):
    def criterion(logit, truth):
        loss = []
        for l,t in zip(logit,truth):
            e = F.cross_entropy(l, t)
            #e = cross_entropy_onehot_loss(l, t)   --- older version
            loss.append(e)

        return loss
    
    valid_loss = np.zeros(6, np.float32)
    valid_num  = np.zeros_like(valid_loss)

    valid_probability = [[],[],[],]
    valid_truth = [[],[],[],]

    for t, (input, truth, infor) in enumerate(valid_loader):

        batch_size = len(infor)

        net.eval()
        input = input.cuda()
        truth = [t.cuda() for t in truth]

        with torch.no_grad():
            logit = data_parallel(net, input) #net(input)
            probability = logit_to_probability(logit)

            loss = criterion(logit, truth)
            correct = metric(probability, truth)

        #---
        loss = [l.item() for l in loss]
        l = np.array([ *loss, *correct, ])*batch_size
        n = np.array([ 1, 1, 1, 1, 1, 1  ])*batch_size
        valid_loss += l
        valid_num  += n

        #---
        for i in range(NUM_TASK):
            valid_probability[i].append(probability[i].data.cpu().numpy())
            valid_truth[i].append(truth[i].data.cpu().numpy())

        #print(valid_loss)
        print('\r %8d /%d'%(valid_num[0], len(valid_loader.dataset)),end='',flush=True)

        pass  #-- end of one data loader --
    assert(valid_num[0] == len(valid_loader.dataset))
    valid_loss = valid_loss/(valid_num+1e-8)

    #------
    for i in range(NUM_TASK):
        valid_probability[i] = np.concatenate(valid_probability[i])
        valid_truth[i] = np.concatenate(valid_truth[i])
    recall, avgerage_recall = compute_kaggle_metric(valid_probability, valid_truth)


    return valid_loss, (recall, avgerage_recall)

In [17]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2
def cutmix(data, targets1, targets2, targets3, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets1 = targets1[indices]
    shuffled_targets2 = targets2[indices]
    shuffled_targets3 = targets3[indices]

    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    data[:, :, bbx1:bbx2, bby1:bby2] = data[indices, :, bbx1:bbx2, bby1:bby2]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))

    targets = [targets1, shuffled_targets1, targets2, shuffled_targets2, targets3, shuffled_targets3, lam]
    return data, targets

def mixup(data, targets1, targets2, targets3, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets1 = targets1[indices]
    shuffled_targets2 = targets2[indices]
    shuffled_targets3 = targets3[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = [targets1, shuffled_targets1, targets2, shuffled_targets2, targets3, shuffled_targets3, lam]

    return data, targets


def cutmix_criterion(preds1,preds2,preds3, targets):
    targets1, targets2,targets3, targets4,targets5, targets6, lam = targets[0], targets[1], targets[2], targets[3], targets[4], targets[5], targets[6]
    criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(preds1, targets1) + (1 - lam) * criterion(preds1, targets2) + lam * criterion(preds2, targets3) + (1 - lam) * criterion(preds2, targets4) + lam * criterion(preds3, targets5) + (1 - lam) * criterion(preds3, targets6)

def mixup_criterion(preds1,preds2,preds3, targets):
    targets1, targets2,targets3, targets4,targets5, targets6, lam = targets[0], targets[1], targets[2], targets[3], targets[4], targets[5], targets[6]
    criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(preds1, targets1) + (1 - lam) * criterion(preds1, targets2) + lam * criterion(preds2, targets3) + (1 - lam) * criterion(preds2, targets4) + lam * criterion(preds3, targets5) + (1 - lam) * criterion(preds3, targets6)
    

In [18]:
def criterion_(logit, truth):
        loss = []
        for l,t in zip(logit,truth):
            e = F.cross_entropy(l, t)
            loss.append(e)

        return loss

In [19]:
def run_train(testing=False,augment=None):

    if testing:
        out_dir = r'C:\Users\Kaggle\BengaliAI\testing'
    else:
        out_dir = r'C:\Users\Kaggle\BengaliAI\seResNext_gridmask_output_contin1'
    initial_checkpoint = r'C:\Users\Kaggle\BengaliAI\seResNext_gridmask_output\checkpoint\00199999_model.pth'

#     schduler = NullScheduler(lr=0.005)
    schduler = CyclicScheduler0(min_lr=0.001, max_lr=0.01, period=10, ratio=1.0)
   
    iter_accum = 1
    batch_size = 16 #8

    ## setup  -----------------------------------------------------------------------------
    for f in ['checkpoint','train','valid'] : os.makedirs(out_dir +'/'+f, exist_ok=True)
        
    log = Logger()
    log.open(out_dir+'/log.train.txt',mode='a')
    log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
    log.write('\t%s\n' % COMMON_STRING)
    log.write('\n')

    log.write('\tSEED         = %u\n' % SEED)
    log.write('\tout_dir      = %s\n' % out_dir)
    log.write('\n')


    ## dataset ----------------------------------------
    log.write('** dataset setting **\n')

    train_dataset = KaggleDataset(
        df = train_df, 
        data_path = data_root,
        augment = train_augment,
    )
    train_loader  = DataLoader(
        train_dataset,
        sampler     = RandomSampler(train_dataset),
        batch_size  = batch_size,
        drop_last   = True,
        num_workers = 0,
        pin_memory  = True,
        collate_fn  = null_collate
    )


    valid_dataset = KaggleDataset(
        df = valid_df, 
        data_path = data_root,
        augment = valid_augment,
    )
    valid_loader = DataLoader(
        valid_dataset,
        sampler     = SequentialSampler(valid_dataset),
        batch_size  = 64,
        drop_last   = False,
        num_workers = 0,
        pin_memory  = True,
        collate_fn  = null_collate
    )

    assert(len(train_dataset)>=batch_size)
    log.write('batch_size = %d\n'%(batch_size))
    log.write('train_dataset : \n%s\n'%(train_dataset))
    log.write('valid_dataset : \n%s\n'%(valid_dataset))
    log.write('\n')

    ## net ----------------------------------------
    log.write('** net setting **\n')
    net = Net().cuda()
    
    
    log.write('\tinitial_checkpoint = %s\n' % initial_checkpoint)

    if initial_checkpoint is not None:
        state_dict = torch.load(initial_checkpoint, map_location=lambda storage, loc: storage)
        net.load_state_dict(state_dict,strict=True)  #True
    else:
        net.load_pretrain(is_print=False)
        print("Loaded pretrained model")


    log.write('net=%s\n'%(type(net)))
    log.write('\n')
    
    
    optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=schduler(0), momentum=0., weight_decay=0.0)

    num_iters   = 200*1000  # comment this for training longer
    iter_smooth = 50
    iter_log    = 250
    iter_valid  = 500
    iter_save   = [0, num_iters-1]\
                   + list(range(0, num_iters, 1000))#1*1000

    start_iter = 0
    start_epoch= 0
    rate       = 0
    if initial_checkpoint is not None:
        initial_optimizer = initial_checkpoint.replace('_model.pth','_optimizer.pth')
        if os.path.exists(initial_optimizer):
            checkpoint  = torch.load(initial_optimizer)
            start_iter  = checkpoint['iter' ]+1    ### critial to load and restart training from previous pt 
            start_epoch = checkpoint['epoch']+1    ### critial to load and restart training from previous pt
        pass

    log.write('optimizer\n  %s\n'%(optimizer))
    log.write('schduler\n  %s\n'%(schduler))
    log.write('\n')

    ## start training here! ##############################################
    log.write('** start training here! **\n')
    log.write('   batch_size=%d,  iter_accum=%d\n'%(batch_size,iter_accum))
    log.write('                    |----------------------- VALID------------------------------------|------- TRAIN/BATCH -----------\n')
    log.write('rate    iter  epoch | kaggle                    | loss               acc              | loss             | time       \n')
    log.write('----------------------------------------------------------------------------------------------------------------------\n')
              #0.01000  26.2  15.1 | 0.971 : 0.952 0.992 0.987 | 0.22, 0.07, 0.07 : 0.94, 0.98, 0.98 | 0.37, 0.13, 0.13 | 0 hr 13 min

    def message(rate, iter, epoch, kaggle, valid_loss, train_loss, batch_loss, mode='print'):
        if mode==('print'):
            asterisk = ' '
            lossl = batch_loss
            
        if mode==('log'):
            asterisk = '*' if iter in iter_save else ' '
            lossl = train_loss
        
        text = \
            '%0.5f %5.1f%s %4.1f | '%(rate, iter/1000, asterisk, epoch,) +\
            '%0.3f : %0.3f %0.3f %0.3f | '%(kaggle[1],*kaggle[0]) +\
            '%4.2f, %4.2f, %4.2f : %4.2f, %4.2f, %4.2f | '%(*valid_loss,) +\
            '%4.2f, %4.2f, %4.2f |'%(*lossl,) +\
            '%s' % (time_to_str((timer() - start_timer),'min'))

        return text


    #----
    kaggle = (0,0,0,0)
    valid_loss = np.zeros(6,np.float32)
    train_loss = np.zeros(3,np.float32)
    batch_loss = np.zeros_like(train_loss)
    iter = 0
    i    = 0



    start_timer = timer()
    while  iter<num_iters:
            
        sum_train_loss = np.zeros_like(train_loss)
        sum_train = np.zeros_like(train_loss)

        optimizer.zero_grad()
        for t, (input, truth, infor) in enumerate(train_loader):
            batch_size = len(infor)
            iter  = i + start_iter
            epoch = (iter-start_iter)*batch_size/len(train_dataset) + start_epoch
            
            #if 0:
            if not testing:
                if (iter % iter_valid==0):
                    valid_loss, kaggle = do_valid(net, valid_loader, out_dir) #
                    pass

                if (iter % iter_log==0):
                    print('\r',end='',flush=True)
                    log.write(message(rate, iter, epoch, kaggle, valid_loss, train_loss, batch_loss,mode='log'))
                    log.write('\n')

                if iter in iter_save:
                    torch.save({
                        #'optimizer': optimizer.state_dict(),
                        'iter'     : iter,
                        'epoch'    : epoch,
                    }, out_dir +'/checkpoint/%08d_optimizer.pth'%(iter))
                    if iter!=start_iter:
                        torch.save(net.state_dict(),out_dir +'/checkpoint/%08d_model.pth'%(iter))
                        pass

            # learning rate schduler -------------
            lr = schduler(iter)
            if lr<0 : break
            adjust_learning_rate(optimizer, lr)
            rate = get_learning_rate(optimizer)

            # one iteration update  -------------
            #net.set_mode('train',is_freeze_bn=True)

            net.train()
            input = input.cuda()
            truth = [t.cuda() for t in truth]
            NUM_CLASS = (168,11,7)
            
    
            logit = data_parallel(net, input)
            probability = logit_to_probability(logit)


            loss = criterion(logit, truth)
            (( 4*loss[0]+loss[1]+loss[2] )/iter_accum).backward()
            
            if (iter % iter_accum)==0:
                optimizer.step()
                optimizer.zero_grad()
            
            
            # print statistics  --------
                
            loss_ = [l.item() for l in loss]
            l = np.array([ *loss_, ])*batch_size
            n = np.array([ 1, 1, 1 ])*batch_size
            batch_loss      = l/(n+1e-8)
            sum_train_loss += l
            sum_train      += n
            if iter%iter_smooth == 0:
                train_loss = sum_train_loss/(sum_train+1e-12)
                sum_train_loss[...] = 0
                sum_train[...]      = 0

            print('\r',end='',flush=True)
            
            print(message(rate, iter, epoch, kaggle, valid_loss, train_loss, batch_loss,mode='print'), end='',flush=True)
            i=i+1

        pass  #-- end of one data loader --
    pass #-- end of all iterations --
    
    log.write('\n')

In [20]:
run_train(testing=False,augment=None)


--- [START 2020-02-12_20-17-27] ----------------------------------------------------------------

	@common.py:  
	set random seed
		SEED = 1581567447
	set cuda environment
		torch.__version__              = 1.1.0
		torch.version.cuda             = 9.0
		torch.backends.cudnn.version() = 7005
		os['CUDA_VISIBLE_DEVICES']     = None
		torch.cuda.device_count()      = 1



	SEED         = 1581567447
	out_dir      = C:\Users\Kaggle\BengaliAI\seResNext_gridmask_output_contin1

** dataset setting **
batch_size = 16
train_dataset : 
	len = 172148


valid_dataset : 
	len = 28692



** net setting **
	initial_checkpoint = C:\Users\Kaggle\BengaliAI\seResNext_gridmask_output\checkpoint\00199999_model.pth
net=<class 'model.Net'>

optimizer
  SGD (
Parameter Group 0
    dampening: 0
    lr: 0.010000000000000002
    momentum: 0.0
    nesterov: False
    weight_decay: 0.0
)
schduler
  CyclicScheduler
min_lr=0.001, max_lr=0.010, period=10.0, ratio=1.00

** start training here! **
   batch_size=16,  it